In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

import anndata
import scanpy as sc
sc.settings.n_jobs = 50
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [2]:
def load_imputation_data(imputation_path, genes=None, cell_types=None, verbose=False):
    '''Load the imputation results into an AnnData object with specified 
    genes and cell types.'''
    # Load the table of gene partitions
    gene_partition_df = pd.read_csv(os.path.join(imputation_path, 'gene_partition.csv')).set_index('gene')
    
    # Define which genes to load
    if genes is None:
        genes = np.array(gene_partition_df.index)
        
    # Define which partitions to load
    partitions = np.unique(gene_partition_df['gene_partition'][gene_partition_df.index.isin(genes)])
        
    # Define which cell types to load
    ct_paths = sorted([f for f in os.listdir(imputation_path) 
                       if os.path.isdir(os.path.join(imputation_path, f))])
    if not (cell_types is None):
        ct_paths = np.intersect1d(ct_paths, [ct.replace('/', '-').replace(' ', '_') for ct in cell_types])
    
    # Load the specified cell types
    adata_list = []
    for ctp in ct_paths:
        adata_ct_list = []
        
        # Load the specified genes
        for p in partitions:
            if verbose:
                print(f"Load {os.path.join(imputation_path, ctp, f'{p}.h5ad')}")
            
            adata_ct_p = anndata.read_h5ad(os.path.join(imputation_path, ctp, f'{p}.h5ad'))
            adata_ct_p = adata_ct_p[:, adata_ct_p.var.index.isin(genes)].copy()
            adata_ct_list.append(adata_ct_p)
          
        adata_list.append(anndata.concat(adata_ct_list, axis=1, merge='first'))
        
    return anndata.concat(adata_list, axis=0)
        

In [3]:
import scipy.spatial
import diffxpy.api
import statsmodels.stats.multitest

def calc_distances_from_cells(query_df, ref_df, slice_col='slice_id', xy_cols=['center_x', 'center_y'],
                             cis=False):
    '''Calculate the distances from the reference cell types.'''

    # Get the coordinates of the query cells and reference cells
    query_coords = np.array(query_df[xy_cols])
    ref_coords = np.array(ref_df[xy_cols])
    
    # Calculate distances from query cells to reference cells slice-by-slice
    all_dists = np.array([np.nan] * query_df.shape[0], dtype=np.float32)    
    slice_ids = np.unique(query_df[slice_col])
    
    for slice_id in slice_ids:
        slice_mask_q = query_df[slice_col] == slice_id
        slice_mask_r = ref_df[slice_col] == slice_id
        
        query_coords_slice = query_coords[slice_mask_q]
        ref_coords_slice = ref_coords[slice_mask_r]
    
        # Skip if there is not enough cells in this slice
        if (len(query_coords_slice) < 2) or (len(ref_coords_slice) < 2):
            continue
            
        # Calculate the distances of K nearest neighbors
        point_tree = scipy.spatial.cKDTree(ref_coords_slice)
        
        if cis:
            distances, indices = point_tree.query(query_coords_slice, k=2)
            distances = distances[:, 1]
        else:
            distances, indices = point_tree.query(query_coords_slice, k=1)
            
        all_dists[slice_mask_q] = distances
        
    return all_dists      
        
def calc_t_test_metrices_for_contact_cells(adata, dist_threshold):
    group_label = np.array(['cell_type0'] * adata.shape[0])
    group_label[adata.obs['dist_to_neighbor_type'] < dist_threshold] = 'cell_type1'
        
    print(f'N contact = {np.sum(group_label == "cell_type1")}')
    print(f'N non-contact = {np.sum(group_label == "cell_type0")}')
        
    test_tt = diffxpy.api.test.t_test(data=adata, grouping=group_label)
    
    return test_tt.summary()

In [4]:
def get_cell_ids_within_dist_range(adata_obs, query_ct, ref_ct, ct_col, min_dist, max_dist, cis=False):
    query_df = adata_obs[adata_obs[ct_col] == query_ct]
    ref_df = adata_obs[adata_obs[ct_col] == ref_ct]
    dists = calc_distances_from_cells(query_df, ref_df, cis=cis)
    
    return np.array(query_df[(dists > min_dist) & (dists < max_dist)].index)

In [5]:
cell_by_region_path = '/home/xingjiepan/data/whole_brain/analysis/20230808_cell_cell_contacts/cells_by_regions'
contact_path = '/home/xingjiepan/data/whole_brain/analysis/20230808_cell_cell_contacts/outputs_30um'


major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

contact_pairs_by_region = {}
all_contact_pairs = []
contact_gene_enrichment_dicts = {}
obs_of_region_dict = {}

for region in major_brain_regions: 
    print(region)
        
    close_contact_subclass_df = pd.read_csv(os.path.join(contact_path, f'{region}_close_contacts.csv'))

    contact_pairs_by_region[region] = [tuple(sorted(p)) for p in 
                                       close_contact_subclass_df[['cell_type1', 'cell_type2']].values]
    
    all_contact_pairs += contact_pairs_by_region[region]
    
    contact_gene_enrichment_dicts[region] = {'subclass_query': [], 'subclass_ref': [], 'gene': [], 
                                'contact_mean': [], 'control_mean': [], 'fold_change': [], 'pval': []}
    
    # Only keep the cells in the region
    obs_of_region_dict[region] = pd.read_csv(os.path.join(cell_by_region_path, f'{region}.csv'), index_col=0)
    
all_contact_pairs = set(all_contact_pairs)

Cerebellum
Cortical_subplate
Fiber_tracts
Hippocampus
Isocortex
Medulla
Olfactory
Pallidum
Pons
Striatum
Thalamus
Ventricular_systems
anterior_HY
anterior_MB
posterior_HY
posterior_MB


In [6]:
# Get the spatially contact cell types
close_contact_subclass_dict = {}

for ct1, ct2 in all_contact_pairs:
    for ct11, ct22 in [(ct1, ct2), (ct2, ct1)]:
        if not (ct11 in close_contact_subclass_dict):
            close_contact_subclass_dict[ct11] = []
        
        if ct22 in close_contact_subclass_dict[ct11]:
            continue
        
        close_contact_subclass_dict[ct11].append(ct22)

In [7]:
imputation_path = '/home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/gene_expression_imputation'
# Load the highly variable genes
import json

with open('subclass_highly_variable_genes.json', 'r') as f:
    hvg_dict = json.load(f)

In [8]:
%%time

contact_range = (0, 30)
control_range = (30, np.float64('inf'))

# Get the gene enrichment in close contact cells
keys = close_contact_subclass_dict.keys()
for key_id, ct_query in enumerate(keys):
    print(f'{key_id}/{len(keys)}, {ct_query}')
    
    # Load the highly imputed highly varible genes for the query cell type
    adata = load_imputation_data(imputation_path, genes=hvg_dict[ct_query], 
                     cell_types=[ct_query], verbose=False)
    
    for ct_ref in close_contact_subclass_dict[ct_query]:
        cis = (ct_query == ct_ref)
            
        for region in major_brain_regions:
            if tuple(sorted([ct_query, ct_ref])) not in contact_pairs_by_region[region]:
                continue
            
            # Get the contact cells and control cells
            contact_cells = get_cell_ids_within_dist_range(obs_of_region_dict[region], ct_query, ct_ref, 
                                    'subclass_label_transfer', contact_range[0], contact_range[1], cis=cis)
        
            control_cells = get_cell_ids_within_dist_range(obs_of_region_dict[region], ct_query, ct_ref, 
                                    'subclass_label_transfer', control_range[0], control_range[1], cis=cis)
            
            # Get the expressions
            adata_contact = adata[adata.obs.index.isin(contact_cells)]
            adata_control = adata[adata.obs.index.isin(control_cells)]

            X_contact = adata_contact.X.toarray()
            X_control = adata_control.X.toarray()
        
            # Calculate the statistics
            contact_means = np.mean(X_contact, axis=0)
            control_means = np.mean(X_control, axis=0)
            fold_changes = contact_means / control_means
            pvals = scipy.stats.ttest_ind(X_contact, X_control, axis=0, 
                                          equal_var=False, alternative='greater')[1]
        
            for i in range(adata_contact.shape[1]):
                contact_gene_enrichment_dicts[region]['subclass_query'].append(ct_query)
                contact_gene_enrichment_dicts[region]['subclass_ref'].append(ct_ref)
            
                contact_gene_enrichment_dicts[region]['gene'].append(adata_contact.var.index[i])
                contact_gene_enrichment_dicts[region]['contact_mean'].append(contact_means[i])
                contact_gene_enrichment_dicts[region]['control_mean'].append(control_means[i])
                contact_gene_enrichment_dicts[region]['fold_change'].append(fold_changes[i])
                contact_gene_enrichment_dicts[region]['pval'].append(pvals[i])
                


0/293, OPC NN
1/293, Oligo NN
2/293, L5/6 IT TPE-ENT Glut
3/293, Sst Gaba
4/293, LSX Sall3 Lmo1 Gaba
5/293, AD Serpinb7 Glut
6/293, Endo NN


<timed exec>:37: RuntimeWarning: invalid value encountered in divide
<timed exec>:37: RuntimeWarning: invalid value encountered in divide


7/293, SC Tnnt1 Gli3 Gaba
8/293, SNc-VTA-RAmb Foxa1 Dopa
9/293, ENTmv-PA-COAp Glut


<timed exec>:37: RuntimeWarning: invalid value encountered in divide


10/293, PVHd-SBPV Six3 Prox1 Gaba
11/293, Peri NN


...

286/293, POR Gata3 Gly-Gaba


<timed exec>:37: RuntimeWarning: divide by zero encountered in divide


287/293, AHN Onecut3 Gaba
288/293, SCs Lef1 Gli3 Gaba
289/293, IRN Dmbx1 Pax2 Gly-Gaba
290/293, OB-mi Frmd7 Gaba
291/293, PBG Mtnr1a Glut-Chol


<timed exec>:37: RuntimeWarning: divide by zero encountered in divide
<timed exec>:37: RuntimeWarning: invalid value encountered in divide


292/293, DMX VII Tbx20 Chol
CPU times: user 1h 23min 1s, sys: 9min 48s, total: 1h 32min 49s
Wall time: 2h 33min 11s


In [9]:
output_path = 'contact_enriched_genes'
os.makedirs(output_path, exist_ok=True)

contact_gene_enrichment_dfs = {}

for region in contact_gene_enrichment_dicts:
    contact_gene_enrichment_dfs[region] = pd.DataFrame(contact_gene_enrichment_dicts[region])
    contact_gene_enrichment_dfs[region] = contact_gene_enrichment_dfs[region][
                                                ~np.isnan(contact_gene_enrichment_dfs[region]['pval'])]
    contact_gene_enrichment_dfs[region]['pval_adjusted'] = statsmodels.stats.multitest.multipletests(
                                            contact_gene_enrichment_dfs[region]['pval'], method='fdr_bh')[1]


    mask = contact_gene_enrichment_dfs[region]['fold_change'] > 2
    mask = mask & (contact_gene_enrichment_dfs[region]['pval_adjusted'] < 0.01)
    contact_gene_enrichment_dfs[region] = contact_gene_enrichment_dfs[region][mask].sort_values(
                                ['pval_adjusted', 'fold_change'], ascending=[True, False])
    
    contact_gene_enrichment_dfs[region].to_csv(os.path.join(output_path, f'{region}_contact_enriched_genes.csv'))

In [10]:
output_path = 'contact_enriched_genes'
major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

contact_gene_enrichment_dfs = {}

for region in major_brain_regions:
    contact_gene_enrichment_dfs[region] = pd.read_csv(os.path.join(
                                                        output_path, f'{region}_contact_enriched_genes.csv'))

In [11]:
pd.concat(contact_gene_enrichment_dfs)

Unnamed: 0           subclass_query  \
Cerebellum   0         329049                  VLMC NN   
             1          69283                  Endo NN   
             2         327917                  VLMC NN   
             3          70372                  Endo NN   
             4          70823                  Endo NN   
...                       ...                      ...   
posterior_MB 6208      144869                   ABC NN   
             6209      497184  PAG-SC Pou4f1 Zic1 Glut   
             6210       89298                   BAM NN   
             6211       88708                   BAM NN   
             6212       54113       SC Tnnt1 Gli3 Gaba   

                              subclass_ref     gene  contact_mean  \
Cerebellum   0                     CHOR NN    Dpep1      3.252998   
             1                      SMC NN    Cytl1      1.504874   
             2                     CHOR NN      Mme      0.966185   
             3                      SMC NN    Fbln5      3.182609   
             4                      SMC NN     Pi16      0.611192   
...                                    ...      ...           ...   
posterior_MB 6208                   SMC NN     Emp1      1.191851   
             6209          SCig Foxb1 Glut  Gm36670      0.132502   
             6210                   ABC NN     Flt3      0.078675   
             6211                   ABC NN     Shc4      0.060459   
             6212  SCig Tfap2b Chrnb3 Glut  Gm13589      0.065839   

                   control_mean  fold_change      pval  pval_adjusted  
Cerebellum   0         0.009311   349.365080  0.000000       0.000000  
             1         0.014590   103.140950  0.000000       0.000000  
             2         0.012198    79.207170  0.000000       0.000000  
             3         0.045910    69.323450  0.000000       0.000000  
             4         0.014933    40.929726  0.000000       0.000000  
...                         ...          ...       ...            ...  
posterior_MB 6208      0.564289     2.112127  0.000962       0.009902  
             6209      0.059608     2.222890  0.000966       0.009940  
             6210      0.017456     4.506964  0.000968       0.009954  
             6211      0.023133     2.613524  0.000970       0.009967  
             6212      0.032680     2.014675  0.000970       0.009970  

[103714 rows x 9 columns]

In [12]:
contact_gene_enrichment_dfs['Isocortex'][
    (contact_gene_enrichment_dfs['Isocortex']['subclass_query'] == 'Vip Gaba')
   &(contact_gene_enrichment_dfs['Isocortex']['subclass_ref'] == 'L2/3 IT CTX Glut')]

,Unnamed: 0,subclass_query,subclass_ref,gene,contact_mean,control_mean,fold_change,pval,pval_adjusted
506,114560,Vip Gaba,L2/3 IT CTX Glut,Col6a3,0.093491,0.030934,3.022236,0.000000e+00,0.000000e+00
595,114545,Vip Gaba,L2/3 IT CTX Glut,Kcne4,0.051471,0.018444,2.790623,0.000000e+00,0.000000e+00
627,114648,Vip Gaba,L2/3 IT CTX Glut,Itih5,0.455746,0.166858,2.731339,0.000000e+00,0.000000e+00
685,115881,Vip Gaba,L2/3 IT CTX Glut,5033421B08Rik,0.474190,0.180798,2.622753,0.000000e+00,0.000000e+00
742,115613,Vip Gaba,L2/3 IT CTX Glut,Tgfbr2,0.120799,0.047794,2.527498,0.000000e+00,0.000000e+00
744,115796,Vip Gaba,L2/3 IT CTX Glut,Ntn1,0.207814,0.082352,2.523488,0.000000e+00,0.000000e+00
749,114584,Vip Gaba,L2/3 IT CTX Glut,Syt2,0.117221,0.046562,2.517502,0.000000e+00,0.000000e+00
770,115506,Vip Gaba,L2/3 IT CTX Glut,Crispld2,0.958992,0.386245,2.482861,0.000000e+00,0.000000e+00
775,114955,Vip Gaba,L2/3 IT CTX Glut,Tpm2,0.249913,0.100759,2.480300,0.000000e+00,0.000000e+00
780,114675,Vip Gaba,L2/3 IT CTX Glut,Fibcd1,0.270682,0.109468,2.472701,0.000000e+00,0.000000e+00


In [13]:
contact_gene_enrichment_dfs['Isocortex'][
    (contact_gene_enrichment_dfs['Isocortex']['subclass_query'] == 'L2/3 IT CTX Glut')
   &(contact_gene_enrichment_dfs['Isocortex']['subclass_ref'] == 'Vip Gaba')]

,Unnamed: 0,subclass_query,subclass_ref,gene,contact_mean,control_mean,fold_change,pval,pval_adjusted


In [14]:
contact_gene_enrichment_dfs['Olfactory'][
            contact_gene_enrichment_dfs['Olfactory']['gene'] == 'Sfrp1']

,Unnamed: 0,subclass_query,subclass_ref,gene,contact_mean,control_mean,fold_change,pval,pval_adjusted
290,412034,Astro-OLF NN,OB-STR-CTX Inh IMN,Sfrp1,1.463033,0.226657,6.454840,0.0,0.0
1811,418316,Astro-OLF NN,Oligo NN,Sfrp1,1.014605,0.366466,2.768617,0.0,0.0
2449,453184,VLMC NN,VLMC NN,Sfrp1,0.370432,0.150858,2.455510,0.0,0.0


In [15]:
contact_gene_enrichment_dfs['Isocortex'][
            contact_gene_enrichment_dfs['Isocortex']['gene'] == 'Pi16']

,Unnamed: 0,subclass_query,subclass_ref,gene,contact_mean,control_mean,fold_change,pval,pval_adjusted
1,46981,Endo NN,SMC NN,Pi16,0.467497,0.010579,44.193108,0.000000e+00,0.000000e+00
349,320205,VLMC NN,SMC NN,Pi16,0.237895,0.066519,3.576334,0.000000e+00,0.000000e+00
1692,52167,Endo NN,VLMC NN,Pi16,0.191532,0.016038,11.942672,2.460720e-202,1.450849e-200
2187,49574,Endo NN,Endo NN,Pi16,0.038719,0.016926,2.287566,3.049870e-92,1.394498e-90
2377,72911,Endo NN,BAM NN,Pi16,0.178484,0.023548,7.579503,2.139483e-63,8.763176e-62
2838,57353,Endo NN,ABC NN,Pi16,0.130049,0.025771,5.046279,1.307796e-25,3.500689e-24
3870,54760,Endo NN,DC NN,Pi16,0.226827,0.027275,8.316235,9.487161e-07,1.109335e-05
5134,78097,Endo NN,Lymphoid NN,Pi16,0.055491,0.026620,2.084580,7.739279e-04,6.032233e-03
